#### [詳解ディープラーニング　TensorFlow・Kerasによる時系列データ処理](https://book.mynavi.jp/ec/products/detail/id=72995)
　巣籠悠輔 著  
  
support site : [https://book.mynavi.jp/supportsite/detail/9784839962517.html](https://book.mynavi.jp/supportsite/detail/9784839962517.html)  
github : [yusugomori/deeplearning-tensorflow-keras](https://github.com/yusugomori/deeplearning-tensorflow-keras) 


---
### (４章　続き)

---
### Weight Initialization
勾配降下法と誤差逆伝播法によって学習を行うとき、  
重みの初期値が等しい各ノードは全く等しい値の誤差を受け取り、等しく更新されるため  
結果として１つのニューロンだけがあるのと同じ振る舞いをしてしまう。  
よって重みの初期値は同じ層のどの２つのノードも異なっていなければならず  
これには初期値の設定をランダム化する手法が使われる。  
  
また、学習中に誤差が全層に正しく伝わり、また値が大きくなり過ぎない、  
つまり不安定勾配問題を軽減するような初期値の分布であることが望ましい。  
  
CS231の講義での[Setting up the data and the model | Weight Initialization](http://cs231n.github.io/neural-networks-2/#init)の項では、経験則が次のようにまとめられている。
  
 - Small random numbers for symmetry breaking
 - Calibrating the variances with 1/sqrt(n) to ensures that all neurons initially have approximately the same output distribution
  

後者で言及されている出力の分布について検討する。  
前提として、入力データは正規化され、重みの分布の平均値が零であること、すなわち

\begin{align*}
E(x_{i}) = E(w_{ij}) = 0
\end{align*}

を仮定する。  
  
ある層への入力が $n$ 次元のベクトル $\mathbf{x}$ 、重みが $\mathbf{W}$ のとき、  
活性化関数へ入力される重み付き和 $\mathbf{p}$ の成分 $ p_{j} = \sum_{i=1}^{n}w_{ij}x_{i}$ の分散を考えると

\begin{align*}
Var(p_{j}) &= Var\left( \sum_{i=1}^{n}w_{ij}x_{i} \right) \\
&= \sum_{i=1}^{n} Var(w_{ij}x_{n}) \\
&= \sum_{i=1}^{n} \left\{ \left( E(w_{ij}) \right)^{2} Var(x_{i}) + \left( E(x_{i}) \right)^{2} Var(w_{ij}) + Var(w_{ij})Var(x_{i}) \right\} \\ 
&= \sum_{i=1}^{n} Var(x_{ij}) Var(w_{i}) & \because E(x_{i}) = E(w_{ij}) = 0\\
&= (n Var(w_{ij}))Var(x_{i})
\end{align*}

となる。  
  
さて、定数 $a$ に対して $Var(ax)=a^{2} Var(x)$ であるから、層の前後で出力分布が変わらないためには  
重みの初期値に係数 $1/ \sqrt{n}$ が乗算されていればよい。  
  


---
代表的な重みの初期値として、以下のようなものがよく知られている。  
ここで fan_in, fan_out は各層（Weight tensor）における入力／出力ユニット数である。

#### LuCunの初期値
[LeCun et al., (1998)](http://yann.lecun.com/exdb/publis/pdf/lecun-98b.pdf)で提唱された初期値で、
正規分布または一様分布による初期化を行う。

Kerasでは

~~~python
# truncated normal distribution | mean=0, stddev = sqrt(1 / fan_in)
initializer = "lecun_normal"

# uniform distribution | within [-sqrt(3 / fan_in), sqet(3 / fan_in)]
initializer = "lecun_uniform"
~~~

により使用できる。

#### Xavierの初期値
[Glorot and Bengio (2010)](http://proceedings.mlr.press/v9/glorot10a.html) で提唱された初期値で  
勾配の大きさを全体として一定に保つことを目的としており、  
前後の層のユニット数から $ Var(w) = \frac{2}{n_{in} + n_{out}}$ の標準偏差をもつ分布を使用するというものである。  
  
使用の際には、一様分布ならば `var = sqrt(6. / (n_in + n_out)); [-x, x]`が、  
正規分布ならば `var = sqrt(3. / (n_in + n_out))` が用いられる。  
  
[TensorFlow](https://www.tensorflow.org/versions/r0.12/api_docs/python/contrib.layers/initializers)では  

~~~python
tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32)
~~~
  
[Keras](https://keras.io/ja/initializers/)では initializerの指定部分で  

~~~python
# normal distribution
initializer = 'glorot_normal'

# uniform distribution
initializer = 'glorot_uniform'
~~~

によって使用できる。

#### Heの初期値
[He et al.(2015)](https://arxiv.org/abs/1502.01852)で提唱された初期値で、活性化関数としてReLUを使う場合の初期値分布である。  
  
Kerasでは  

~~~python
# trancated normal distribution | mean=0, stddev = sqrt(2/fan_in)
initializer = "he_normal"

# uniform distribution | within [-sqrt(6 / fan_in), sqrt(6 / fan_in)]
initializer = "hi_uniform"
~~~

によって使用できる。  

---
### Optimizer
　よく使われる最適化手法は勾配降下法とその改良手法である。  
　そのうちいくつかを選んで記載する。  

#### （先に結論）  
　どうも定説がなく数カ月ごとに主流の学説が入れ替わるような状況であるが、  
　特段の理由がない限りSGDまたはMomentumで問題ないと思われる。    
   
#### （関連資料）  
[On Large-Batch Training for Deep Learning: Generalization Gap and Sharp Minima](https://arxiv.org/abs/1609.04836)  
　minibatch(~512)がlarge batchよりも上手く行く理由の検証。  
　小さなbatchは周辺がflatな最適解に向かう傾向が、large batchはsharpな最適解に向かう傾向がある。  
　（より小さいbatchの使用は更新量の分散を大きくするために、局所解に留まることを難しくするようだ）  
  
[Train longer, generalize better: closing the generalization gap in large batch training of neural networks](https://arxiv.org/abs/1705.08741)  
　flatな最適解を得るためには更新回数が重要で、large batchでも回数を増やすことや、  
　学習率を高くする、Batch Normalizationの適用で汎化性能を高めることができる。  
  
[Understanding deep learning requires rethinking generalization](https://openreview.net/forum?id=Sy8gdB9xx&noteId=Sy8gdB9xx)   
　DNNは全てのラベルを覚えきる能力を持ちながら正則化では説明のつかない汎化を示し、  
　SGDがその要因のひとつである。  
  　  
[The Marginal Value of Adaptive Gradient Methods in Machine Learning](https://arxiv.org/abs/1705.08292v1)  
　RMSprop, Adamなどの局所的な曲率情報に基づく手法は単純なSGDやmomentum SGDに比べ過学習しやすく、  
　収束速度は改善しない。  
  
[Fixing Weight Decay Regularization in Adam](https://arxiv.org/abs/1711.05101)  
　AdamがSGDに比べ汎化性能が低いのはWeight Decay regularizationが意図した値になっていないのが原因であり、  
　学習率への依存を無くしてやると性能改善する  


#### この節を記述する際の参考資料：    
　[arXivTimes](https://github.com/arXivTimes/arXivTimes)   
　[Optimizer : 深層学習における勾配法について](https://qiita.com/tokkuman/items/1944c00415d129ca0ee9)  
　[OPTIMIZER 入門 ~線形回帰からAdamからEveまで](https://qiita.com/deaikei/items/29d4550fa5066184329a)  
　[An overview of gradient descent optimization algorithms](http://ruder.io/optimizing-gradient-descent/)  

・ノルム  
　$n$次元ベクトル $\mathbf{x} = (x_{1}, x_{2}, \dots, x_{n})$ および $ 1 \leq p < \infty $ なる $p$ に対して  

\begin{align*}
\sqrt[p]{ |x_{1}|^{p} + |x_{2}|^{p} + \cdots + |x_{n}|^{p}}
\end{align*}

　を $\mathbf{x}$ の $L^{p}$ ノルムといい、 $||\mathbf{x}||_{p}$ と書く。 
  
　いわゆる「距離」は$L^{2}$ ノルムであり、ユークリッドノルムという。  
　機械学習では $L^{1}$ ノルム、$L^{2}$ ノルム、$L^{\infty}$ ノルムがよく用いられる。  

---
#### Optimization Methods

・勾配降下法（再掲）  
　現実的な最適化問題では解析的に解けることは少なく、その場合に用いられる手法の一つが勾配(降下)法である。  
　その表現の１つは、何らかの意味で正解からの離れ具合を示す関数 $f$ および学習率 $\eta$ を定めて、  
 
\begin{align*}
x_{i+1} = x_{i} - \eta \frac{\partial f}{\partial x_{i}}
\end{align*}

　によって値 $x$ を更新していくものである。  
  
　本手法は機械学習では Batch gradient descent として実装されており、  
　訓練データ全体に対する計算が必要であることから遅く、実用上より良いOptimizerが開発されてきた。    
　SGDおよび他のよく使われる手法の多くは勾配降下法の改良版である。 
  
　以下、記号については別記なき限り、  
　重みなどモデルのパラメータを $\theta$ 、誤差関数を $C$ 、誤差関数の $\theta$ に対する勾配を $\nabla_{\theta}$ 、学習率を $\eta$ 、  
　訓練データと正解（ラベル）データをそれぞれ $X = \{ x_{i} \}, Y = \{ y_{i} \}$ とする。  


### SGD (Stochastic Gradient Descent) : オンライン勾配降下法およびミニバッチ勾配降下法
　データセットよりミニバッチ $(x^{(i)}, y^{(i)})$ をランダムに抽出し、

\begin{align*}
\theta = \theta - \eta \nabla_{\theta} C(\theta; x^{(i)}; y^{(i)})
\end{align*}

　によってパラメータを更新する。  
  
### Momentum SGD
　SGDによる学習時にパラメータが最適解の周辺で振動して学習が進まない問題を解決するため  
　慣性の考え方を導入した Optimizer である。  
　モメンタム項 $v$ およびその係数 $\gamma \, (<1) $ を導入して、  
 
\begin{align*}
v_{t} &= \gamma v_{t-1} + \eta \nabla_{\theta} C(\theta) \\
\theta &= \theta - v_{t}
\end{align*}

　によりパラメータを更新する。    
　モメンタム項により、複数ステップに渡って勾配が同じ方向に向くとき  
　学習はこの方向、つまり局所解のある方向に向けて加速する。    
  
### Nesterov accelerated gradient (Nesterov Momentum)
　Momentumに先を予測する能力を持たせたもので、次のパラメータの位置を $\theta -\gamma v_{t-1}$ によって近似し、
 
\begin{align*}
v_{t} &= \gamma v_{t-1} + \eta \nabla_{\theta} C(\theta -\gamma v_{t-1}) \\
\theta &= \theta - v_{t}
\end{align*}

　によって更新する。  

### AdaGrad
　適応的な学習率を持つOptimizerで、稀なパラメータに対してより大きな更新をさせるために  
　スパースなデータを扱うのに適する。  

　Adagradでは、各パラメータ $\theta_{t, i}$ について、対角成分 $(i,i)$ がタイムステップ $t$ までの $\theta_{i}$ に対する勾配の二乗和であるような  
　対角行列 $G_{t} \in \mathbb{R}^{d \times d}$ および平滑化項 $\epsilon$ （通常 1e-8 程度）を用いて  

\begin{align*}
G_{t, ii} = \sum_{step=0}^{t} \left( \Delta_{\theta}C(\theta_{step,i}) \right)^{2} \\
\theta_{t+1, i} = \theta_{t, i} - \frac{\eta}{\sqrt{G_{t, ii} + \epsilon }} \cdot \nabla_{\theta} C(\theta_{i})
\end{align*}

　の更新規則によって最適化を行う。これをベクトル化したものは次のようになる。
 
\begin{align*}
\theta_{t+1} = \theta_{t} - \frac{\eta}{\sqrt{G_{t} + \epsilon }} \odot \nabla_{\theta} C(\theta)
\end{align*}

　Adagradは手動で学習率を調整する必要がなく扱いやすい。  
　一方で分母の二乗勾配が蓄積するという弱点があり、学習率が単調減少して小さくなりすぎる傾向にある。  
  
### AdaDelta
　Adagradの発展形で、学習率の急速な単調減少を防ぐために改良された。  
　勾配の蓄積を減衰するため、Adagradの更新規則にタイムステップ $t$ における移動平均 $E[g^{2}]_{t}$ を導入して  
 
\begin{align*}
\theta_{t+1} = \theta_{t} - \frac{\eta}{\sqrt{E[g^{2}]_{t} + \epsilon }} \odot \nabla_{\theta} C(\theta)
\end{align*}

　と変更する。  
　この分母は勾配についての RMS (二乗平均平方根) 誤差基準であるから、これを省略して
 
\begin{align*}
\theta_{t+1} = \theta_{t} - \frac{\eta}{RMS[g]_{t}} \odot \nabla_{\theta} C(\theta)
\end{align*}

　と書ける。  
   
　ただし、この更新式において単位が一致しないことから、元の論文ではパラメータと同じ単位を  
　仮定的に持たせるためにパラメータ更新の二乗についての減衰平均を先に定義している。  
 
\begin{align*}
E[\Delta \theta^{2}]_{t} &= \gamma E[\Delta \theta^{2}]_{t-1} + (1 - \gamma) \Delta \theta^{2}_{t} \\
RMS[\Delta\theta]_{t} &= \sqrt{E[\Delta\theta^{2}]_{t} + \epsilon}
\end{align*}

　未知である $RMS[\Delta\theta]_{t}$ を前のタイムステップまでのパラメータ更新のRMSで近似して、  
　これによって学習率 $\eta$ を置換して次の更新規則を得る。  
 
\begin{align*}
\Delta \theta_{t} &= - \frac{RMS[\Delta\theta]_{t-1}}{RMS[g]_{t}}g_{t} \\
\theta_{t+1} &= \theta_{t} + \Delta \theta_{t}
\end{align*}

　この後者の更新規則では学習率の項を含まないため、学習率の初期値を与える必要がない。  
   
　TensorFlowなどの実装では、上記の更新規則を  
 
\begin{align*}
\theta_{t+1} &= \theta_{t} + \alpha \Delta \theta_{t}
\end{align*}

　としており、引数として与えたlearning rateは $\alpha$ に代入される。  

### RMSprop
　Adadeltaとは独立に編み出されたAdagradの改良版で、その更新規則は

\begin{align*}
E[g^{2}]_{t} &= 0.9 E[g^{2}]_{t-1} + 0.1 g^{2}_{t} \\
\theta_{t+1} &= \theta_{t} - \frac{\eta}{\sqrt{E[g^{2}]_{t} + \epsilon}} g_{t}
\end{align*}

　となり、Adadeltaのベクトル更新の式で $\gamma = 0.9$ とした場合と同じである。  
  

### Adam (Adaptive Moment Estimation)
　AdadeltaやRMSpropで使われた過去の勾配の２乗 $v_{t}$ の指数関数的減衰平均に加え、  
　過去の勾配 $m_{t}$ の指数関数的減衰平均を保持しする。  
　$m_{t}, v_{t}$ はそれぞれ勾配の一次モーメントと二次モーメントの概算値である。  

\begin{align*}
m_{t} &= \beta_{1}m_{t-1} + (1-\beta_{1})g_{t} \\
v_{t} &= \beta_{2}v_{t-1} + (1-\beta_{2})g_{t}^{2}
\end{align*}

　これらのモーメントの偏りをバイアス補正した推定値を用いてパラメータを更新する。    

\begin{align*}
\hat{m_{t}} &= \frac{m_{t}}{1-\beta^{t}_{1}} \\
\hat{v_{t}} &= \frac{v_{t}}{1-\beta^{t}_{2}} \\
\theta_{t+1} &= \theta_{t} - \frac{\eta}{\sqrt{\hat{v_{t}}} + \epsilon}\hat{m_{t}}
\end{align*}

　元論文では初期値として、$\beta_{1}=0.9, \beta_{2}=0.999, \epsilon=10^{-8}$ が提案されている。
 
### AdaMax
　Kingma and Ba (2015) によって提案された Optimizer である。  
　Adamの更新式では、 $v_{t}$ の項は勾配を過去の勾配（ $v_{t-1}$ に含まれる）と  
　現在の勾配の $l_{2}$ ノルムに反比例してscaleさせる。  

\begin{align*}
v_{t} &= \beta_{2}v_{t-1} + (1-\beta_{2})|g_{t}|^{2}
\end{align*}

　これを $l_{p}$ ノルムについて一般化すると次のようになる。

\begin{align*}
v_{t} &= \beta_{2}^{p}v_{t-1} + (1-\beta_{2}^{p})|g_{t}|^{p}
\end{align*}

　一般に大きな $p$ に対するノルムは不安定であり $l_{1}$、$l_{2}$ が実践でよく用いられるが、  
　$l_{\infty}$ ノルムも概して安定な振舞いを示す。  
　著者らの報告では、$p=\infty$ としたときの $v_{t}$ を改めて $u_{t}$ とおいた  
 
\begin{align*}
u_{t} &= \beta_{\infty}^{p}v_{t-1} + (1-\beta_{2}^{\infty})|g_{t}|^{\infty} \\
&= max(\beta_{2} \cdot v_{t-1}, |g_{t}|)
\end{align*}

　はより安定した振舞いを見せた。  
　これを用いて、AdaMax の更新式は次のように書ける。  
 
\begin{align*}
\theta_{t+1} = \theta_{t} - \frac{\eta}{u_{t}}\hat{m_{t}}
\end{align*}

　提案されている値は $\eta=0.002, \beta_{1}=0.9, \beta_{2}=0.999$ である。  


---
### Optimizer の使用法  
  
TensorFlowとKerasに実装されている[Optimizer](https://www.tensorflow.org/api_guides/python/train#Optimizers)は次の通り。  
Kerasではインスタンスをmodel.compile(optimizer=[optimizer instance])で渡すほか、名前で指定することもできる。  

|Methods|TensorFlow Optimizers|Keras Optimizers|
|:--|:--|:--|
|SGD|tf.train.GradientDescentOptimizer|keras.optimizers.SGD|
|Momentum|tf.train.MomentumOptimizer|keras.optimizers.SGD|
|Nesterov Momentum|tf.train.MomentumOptimizer|keras.optimizers.SGD|
|Adagrad|tf.train.AdagradOptimizer|keras.optimizers.Adagrad|
|Adagrad Dual Averaging|tf.train.AdagradDAOptimizer||
|Adadelta|tf.train.AdadeltaOptimizer|keras.optimizers.Adadelta|
|Adam|tf.train.AdamOptimizer|keras.optimizers.Adam|
|Adamax| |keras.optimizers.Adamax|
|Nadam| |keras.optimizers.Nadam|
|FTRL|tf.train.FtrlOptimizer||
|Proximal GD|tf.train.ProximalGradientDescentOptimizer||
|Proximal Adagrad|tf.train.ProximalAdagradOptimizer||
|RMSProp|tf.train.RMSPropOptimizer|keras.optimizers.RMSprop|



---
### Early Stopping
学習回数を過剰に増やした場合、訓練データでの誤差は減少するものの、  
検証データでの誤差が増加する（Overfitting）ことが知られている。  
  
<**注**>：  
　2016年末ごろまでは主流の説でしたが、2017年11月現在では意見が割れています。  
　Early Stoppingを考えなしに本番用モデルへ導入することは推奨できません。  
　単なる P hacking に終わる恐れがあります。  
  
Overfittingを防ぐため、何らかの基準を設けて学習を早期終了させるのがEarly Stoppingであり  
予測精度の改善が一定回数連続して規定値を下回った場合に学習を終了させるという実装が多い。  
  
Tensorflowでは各エポックの最後などに判定処理を実装する。  
Kerasではコールバックとして実装されており、手軽に導入できる。  
  
本notebookでは実装は省略する。  

---
### Batch Normalization
ミニバッチ学習では、例え訓練データを白色化している場合でも、ミニバッチ間で分布に差が生じる。  
これをミニバッチ毎に正規化するのが Batch Normalization である。  
  
　参考：  
　・[Batch Normalization の理解](https://qiita.com/t-tkd3a/items/14950dbf55f7a3095600)  
　・[Understanding the backward pass through Batch Normalization Layer](https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html)  
  

実装時はlayerとして用意した上で、学習可能なパラメータとして scale = $\gamma$ , shift = $\beta$ を与え、  
ミニバッチ $B=\{x_{i} \, | i=1, \ldots, m\}$ について

\begin{align*}
mean : \, \mu_{B} &=\frac{1}{m} \sum_{i=1}^{m} x_{i} \\
var : \, \sigma^{2}_{B} &= \frac{1}{m}\sum_{i=1}^{m}(x_{i}-\mu_{B})^{2}
\end{align*}

を求め、各データ $x_{i}$ を次のように正規化して $\{ y_{i} \}$ を出力する。  

\begin{align*}
\hat{x_{i}} &= \frac{x_{i} - \mu_{B}}{\sqrt{\sigma^{2}_{B} + \epsilon}} \\
y_{i} &= \gamma \hat{x_{i}} + \beta
\end{align*}
  

Batch Normalization layerの勾配を求める。
誤差関数を $C$ として

\begin{align*}
\frac{\partial C}{\partial \gamma} &= \sum_{i=1}^{m} \frac{\partial C}{\partial y_{i}} \frac{\partial y_{i}}{\partial \gamma} \\
&= \sum_{i=1}^{m} \frac{\partial C}{\partial y_{i}} \cdot \hat{x_{i}} \\
\mbox{} \\
\frac{\partial C}{\partial \beta} &= \sum_{i=1}^{m} \frac{\partial C}{\partial y_{i}} \frac{\partial y_{i}}{\partial \beta} \\
&= \sum_{i=1}^{m} \frac{\partial C}{\partial y_{i}} \\
\mbox{} \\
\frac{\partial C}{\partial x_{i}} &= \frac{\partial C}{\partial \hat{x_{i}}} \frac{\partial \hat{x_{i}}}{\partial x_{i}} + \frac{\partial C}{\partial \sigma^{2}_{B}} \frac{\partial \sigma^{2}_{B}}{\partial x_{i}} + \frac{\partial C}{\partial \mu_{B}} \frac{\partial \mu_{B}}{\partial x_{i}} \\
&= \frac{\partial C}{\partial x_{i}} \cdot \frac{1}{\sqrt{\sigma^{2}_{B} + \epsilon}} + \frac{\partial C}{\partial \sigma^{2}_{B}} \cdot \frac{2(x_{i} - \mu_{B})}{m} + \frac{\partial C}{\partial \mu_{B}} \cdot \frac{1}{m}
\end{align*}

となる。右辺の各勾配は

\begin{align*}
\frac{\partial C}{\partial y_{i}} &= Backpropagated \, Error \\
\mbox{} \\
\frac{\partial C}{\partial \hat{x_{i}}} &= \frac{\partial C}{\partial y_{i}} \frac{\partial y_{i}}{\partial \hat{x_{i}}} \\
&= \frac{\partial C}{\partial y_{i}} \cdot \gamma \\
\mbox{} \\
\frac{\partial C}{\partial \sigma^{2}_{B}} &= \sum_{i=1}^{m} \frac{\partial C}{\partial x_{i}} \frac{\partial x_{i}}{\sigma^{2}_{B}} \\
&= \sum_{i=1}^{m} \frac{\partial C}{\partial \hat{x_{i}}} \cdot (x_{i} - \mu_{B}) \cdot \frac{-1}{2} (\sigma^{2}_{B} + \epsilon)^{- \frac{3}{2}} \\
\mbox{} \\
\frac{\partial C}{\partial \mu_{B}} &= \sum_{i=1}^{m} \frac{\partial C}{\partial \hat{x_{i}}} \frac{\partial \hat{x_{i}}}{\partial \mu_{B}} + \frac{\partial C}{\partial \sigma^{2}_{B}} \frac{\partial \sigma^{2}_{B}}{\partial \mu_{B}} \\
&= \sum_{i=1}^{m} \frac{\partial C}{\partial \hat{x_{i}}} \cdot \frac{-1}{\sqrt{\sigma^{2}_{B} + \epsilon}} + \sum_{i=1}^{m} \frac{\partial C}{\partial \sigma^{2}_{B}} \cdot \frac{-2 (x_{i} - \mu_{B})}{m}
\end{align*}

となるので、勾配を求めることができる。  
また、バイアス項は正規化処理によって消える。  

---
#### 実装
TensorFlow, KerasともAPIで用意されているので使用は簡単である。  
  
TensorFlowでは [tf.contrib.layers.batch_norm](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/batch_norm) や [tf.nn.batch_normalization](https://www.tensorflow.org/api_docs/python/tf/nn/batch_normalization)により、  
一例としては、モデル中の適当な場所に以下のようなコードを挟めばよい。  


```python
h = tf.contrib.layers.batch_norm(input_h, center=True, scale=True, is_training=phase)

# 中略　後半はgammaとbetaをupdateするのに必要

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = optimizer.minimize(loss)
```

別の例では

```python
Wn = tf.Variable(Wn_initial)
Zn = tf.matmul(h[n-1], Wn)
batch_mean, batch_var = tf.nn.moments(input_x, axes=[0])

gamma_n = tf.Variable()
beta_n = tf.Variable()

h[n] = tf.nn.batch_normalization(Zn, batch_mean, batch_var, beta_n, scale_n, epsilon)

```

Kerasでは

```python
model.add(BatchNormalization())
```
  
となる。  
  
（参考）：  
　・[Implementing Batch Normalization in Tensorflow](https://r2rt.com/implementing-batch-normalization-in-tensorflow.html)  
　・[TENSORFLOW GUIDE: BATCH NORMALIZATION](http://ruishu.io/2016/12/27/batchnorm/)  
   

---
WIP

---
### Batch Normalization のAPIによらない実装

---
#### 脱線：生物の神経系における正規化の例
生物の神経においても正規化に相当する情報処理が見つかっており、  
例えば [S.Dasgupta et al.(2017) "A neural algorithm for a fundamental computing problem"](http://science.sciencemag.org/content/358/6364/793) は、  
ハエの嗅覚野にてLSH（Locality-Sensitive Hashing）に相当する処理が行われていると主張している。  
 - １層目：臭いの入力を、異なる臭いの種類と濃度に対して同じ平均を持つように正規化した発火率の指数分布に変換  
 - ２層目：入力50から出力2000へ、疎かつ二値のランダム射影  
 - ３層目：APLから抑制性フィードバックを受け、Winner Takes All 則により上位5%のみを残す。これがhash値として働く  
  
Table 1. The fenerality of locality-sensitive hashing in the brain.:

||Step 1|Random projection|Step 2 (expansion)|Step 3 (WTA)|
|:--|-:-|-:-|-:-|
|Fly olfaction|Antennae lobe; 50 glomeruli|Sparse, binary: samples siz glomeruli|Mushroom body; 2000 Kenyon cells|APL neuron; top 5%|
|Mouse olfaction|Olfactory bulb; 1000 glomeruli|Dense, weak: sampes all glomeruli|Piriform cortex: 100,000 semi-lunar cells|Layer 2A: top 10%|
|Rat cerebellum|Precerebellar nuclei|Sparse, binary: samples four precereellar nuceli|Granule cell layer: 250 million granule cells|Golgi cells: top 10 to 20%|
|Rat hippocampus|Entorhinal cortes: 30,000 grid cells | Unknown| Dentate gyrus: 1.2 million granule cells|Hilar cells: top 2%|
  
Quoted from S.Dasgupta et al(2017)  